In [17]:
import anthropic
import pandas as pd
import os
import base64

client = anthropic.Anthropic()

In [18]:
csv_path_natural = f"data/natural/affordance_natural.csv"
img_folder_natural = f"data/natural/images"

csv_path_synthetic = f"data/synthetic/affordance_synthetic.csv"
img_folder_synthetic = f"data/synthetic/images"

df_natural = pd.read_csv(csv_path_natural)
df_synthetic = pd.read_csv(csv_path_synthetic)

In [21]:
!curl https://api.anthropic.com/v1/messages \
     --header "x-api-key: $KEY_HERE" \
     --header "anthropic-version: 2023-06-01" \
     --header "content-type: application/json" \
     --data \
'{"model": "claude-3-5-sonnet-20240620","max_tokens": 20,"messages": [{"role": "user", "content": "Hello, world"}] }'

{"type":"error","error":{"type":"invalid_request_error","message":"Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits."}}

In [12]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Function to iterate and encode images in a folder
def encode_images_in_folder(folder_path):
    encoded_images = {}
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        if os.path.isfile(image_path):
            encoded_images[image_name] = encode_image(image_path)
    return encoded_images

encoded_natural_images = encode_images_in_folder(img_folder_natural)
encoded_synthetic_images = encode_images_in_folder(img_folder_synthetic)

In [15]:
def generate_data_source(base64_image, image_name):
    if image_name.lower().endswith('.png'):
        mime_type = 'image/png'
    else:
        mime_type = 'image/jpeg'
    
    return {"type": "base64", "media_type": mime_type, "data": base64_image}

def claude_vision_call(prompt, base64_image, image_name):
    data_source = generate_data_source(base64_image, image_name)
    
    message = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=3,
        temperature = 0,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": data_source,
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ],
            }
        ],
    )
    return message

def iterate_claude(df, encoded_images_dict):
    results_afforded = []
    results_non_afforded = [] 
    results_related = []
    
    prompt_text = """In this task, you will read short passages and look at an image of an object.
Please rate how sensible it would be to take the action described in the last sentence using the object in
the image in the context of the whole passage. The scale goes from 1 (virtual nonsense) to 7 (completely sensible).
Be sure to read the sentences carefully. Please respond only with a number between 1 and 7."""

    for _, row in df.iterrows():
        condition = row['condition']
        
        for image_type in ['afforded_image', 'non_afforded_image', 'related_image']:
            image_name = row[image_type]
            if image_name in encoded_images_dict:
                base64_image = encoded_images_dict[image_name]
                response = claude_vision_call(prompt_text, base64_image, image_name)
            else:
                response = "Image encoding not found"
            
            if image_type == 'afforded_image':
                results_afforded.append(response)
            elif image_type == 'non_afforded_image':
                results_non_afforded.append(response)
            elif image_type == 'related_image':
                results_related.append(response)
    
    df['claude_result_afforded'] = results_afforded
    df['claude_result_non_afforded'] = results_non_afforded 
    df['claude_result_related'] = results_related

    return df

In [16]:
df_natural = iterate_claude(df_natural, encoded_natural_images)
df_synthetic = iterate_claude(df_synthetic, encoded_synthetic_images)

TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"

In [ ]:
# message = client.messages.create(
#     model="claude-3-5-sonnet-20240620",
#     max_tokens=3,
#     temperature = 0,
#     messages=[
#         {
#             "role": "user",
#             "content": [
#                 {
#                     "type": "image",
#                     "source": {
#                         "type": "base64",
#                         "media_type": image1_media_type,
#                         "data": image1_data,
#                     },
#                 },
#                 {
#                     "type": "text",
#                     "text": """In this task, you will read short passages and look at an image of an object.
# Please rate how sensible it would be to take the action described in the last sentence using the object in
# the image in the context of the whole passage. The scale goes from 1 (virtual nonsense) to 7 (completely sensible).
# Be sure to read the sentences carefully. Please respond only with a number between 1 and 7."""
#                 }
#             ],
#         }
#     ],
# )
# print(message)